# Lab 10.9.5: IMDb Document Classification

## Attribution
This notebook follows lab 10.9.5 from ISLRv2. The R-code has been ported to Python by Daniel Kapitan (23-01-2022).


## Data preparation
We perform document classification on the IMDb dataset, which is available as part of the `tensorflow.keras` package. We limit the dictionary size to the 10,000 most frequently-used words and tokens.

In [1]:
import itertools
import os
import pickle
from pprint import pprint

import altair as alt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


# let's keep our keras backend tensorflow quiet
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# load the data
MAX_FEATURES = 10_000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=MAX_FEATURES)

2022-01-30 15:50:57.967465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-30 15:50:57.967504: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


The last line is a shortcut for unpacking the list of lists. Each element of `X_train` is a vector of numbers between 0 and 9999 (the document), referring to the words found in the dictionary. For example, the first training document is the positive review on page 419 of ISLRv2. The indices of the first 12 words are given below.

In [2]:
X_train[1][:12]

[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012]

To see the words, we create a function, `decode_review()`, that provides a simple interface to the dictionary.

In [3]:
word_index = imdb.get_word_index()

def decode_review(text, word_index=word_index, start_char=1, oov_char=2, index_from=3):
    """Decodes one-hot encoded reviews from IMDb.
    
    Note default values for `imdb.load_data`, see https://keras.io/api/datasets/imdb/
    """
    reverse_word_index = {v + index_from: k for k, v in word_index.items()}

    # add special tags
    tags = {0: "<PAD>", start_char: "<START>", oov_char: "<UNK>"}
    reverse_word_index = {**tags, **reverse_word_index}

    return " ".join([reverse_word_index.get(i, "<index not found>") for i in text])


decode_review(X_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

## Using spare binary matrices

Next we write a function to "one-hot" encode each document in a list of documents, and return a binary matrix in sparse-matrix format.

In [14]:
def one_hot_encode(sequences, dimension=MAX_FEATURES):
    """ One-hot encodes IMDb reviews as SciPy sparse matrix.
    
    Using csr_matrix, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
    For more on sparse matrices, see https://machinelearningmastery.com/sparse-matrices-for-machine-learning/
    """
    seqlen = [len(sequence) for sequence in sequences]
    n = len(seqlen)
    row_index = np.repeat(range(n), seqlen)
    col_index = list(itertools.chain(*sequences))
    data = np.ones(len(row_index), dtype="int")
    return csr_matrix((data, (row_index, col_index)), shape=(n, dimension))

To construct the matrix, one supplies just the entries that are nonzero. In the last line we call [`csr_matrix()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) and supply the row indices corresponding to each document and the column indices corresponding to the words in each document. `data` is literally a list of ones. Words that appear more than once in any given document still get recorded as one.

In [5]:
X_train_1h = one_hot_encode(X_train)
X_test_1h = one_hot_encode(X_test)
y_train = np.array(y_train).astype("float32")
y_test = np.array(y_test).astype("float32")

X_train_1h.shape

(25000, 10000)

In [6]:
# TO DO: don't understand why we get 2.3% non zeros
X_train_1h.sum().sum() / (25_000 * 10_000)

0.023871364

Only 1.3% of the entries are nonzero, so this amounts to considerable savings in memory. We create a validation set of size 2,000, leaving 23,000 for training.

## Fit Lasso Logistic Regression

First we fit a lasso logistic regression model using `sklearn.linear_model.LogisticRegression()` on the training data, and evaluate its performance on the validation data. In order to plot the accuracy as a function of the shrinkage parameter $\lambda$, we iterate over different values of $-log \lambda = [1, 2, .. 20]$. Note that `sklearn` uses `C` as the regularization parameter, which is the inverse of lambda. Similar espressions compute the performance on the test data, and were used to produce the left plot in Figure 10.11. The code takes advantage of the sparse-matrix format of `x_train_1h`and runs in about two minutes; in the usual dense format it would take about over orders of magnitude longer.

In [7]:
np.random.seed(3)
ival = sorted(np.random.choice(range(len(y_train)), 2000))
mask = np.ones_like(y_train, dtype=bool)
mask[ival] = False

In [8]:
%%time
models = {}
for log_lambda in range(1, 20):
    C = 1 / (2 ** ((log_lambda / 2)))
    print(f"fitting LogisticRegression with C = {C:.3f}")
    models[C] = LogisticRegression(C=C, max_iter=1000, random_state=0).fit(
        X_train_1h[mask, :], y_train[mask]
    )
    print("... done")

fitting LogisticRegression with C = 0.707
... done
fitting LogisticRegression with C = 0.500
... done
fitting LogisticRegression with C = 0.354
... done
fitting LogisticRegression with C = 0.250
... done
fitting LogisticRegression with C = 0.177
... done
fitting LogisticRegression with C = 0.125
... done
fitting LogisticRegression with C = 0.088
... done
fitting LogisticRegression with C = 0.062
... done
fitting LogisticRegression with C = 0.044
... done
fitting LogisticRegression with C = 0.031
... done
fitting LogisticRegression with C = 0.022
... done
fitting LogisticRegression with C = 0.016
... done
fitting LogisticRegression with C = 0.011
... done
fitting LogisticRegression with C = 0.008
... done
fitting LogisticRegression with C = 0.006
... done
fitting LogisticRegression with C = 0.004
... done
fitting LogisticRegression with C = 0.003
... done
fitting LogisticRegression with C = 0.002
... done
fitting LogisticRegression with C = 0.001
... done
CPU times: user 7min 7s, sys: 1

In [9]:
accuracy = [
    (
        (k, "train", accuracy_score(y_train[mask], v.predict(X_train_1h[mask, :]))),
        (
            k,
            "validation",
            accuracy_score(y_train[ival], v.predict(X_train_1h[ival, :])),
        ),
    )
    for k, v in models.items()
]
df_lr = pd.DataFrame(itertools.chain(*accuracy), columns=["C", "fold", "accuracy"])
plot_lr = (
    alt.Chart(df_lr)
    .mark_line(point=True)
    .encode(x="log_C:Q", y="accuracy", color="fold")
    .transform_calculate(log_C="log(datum.C)")
)
plot_lr

alt.Chart(...)

## Build a two-layer feedforward network

In [10]:
# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(10_000,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


2022-01-30 15:52:30.954388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-30 15:52:30.954466: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-30 15:52:30.954484: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xps): /proc/driver/nvidia/version does not exist
2022-01-30 15:52:30.954748: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
%%time
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
history = model.fit(X_train_1h[mask,:], y_train[mask],
          batch_size=512, epochs=20,
          verbose=2,
          validation_data=(X_train_1h[ival,:], y_train[ival]))

Epoch 1/20


/home/dkapitan/.pyenv/versions/anaconda3-2021.11/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 16), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


46/46 - 1s - loss: 0.4849 - accuracy: 0.7867 - val_loss: 0.3485 - val_accuracy: 0.8660 - 1s/epoch - 23ms/step
Epoch 2/20
46/46 - 1s - loss: 0.2904 - accuracy: 0.8960 - val_loss: 0.3154 - val_accuracy: 0.8820 - 510ms/epoch - 11ms/step
Epoch 3/20
46/46 - 1s - loss: 0.2308 - accuracy: 0.9173 - val_loss: 0.2743 - val_accuracy: 0.8960 - 505ms/epoch - 11ms/step
Epoch 4/20
46/46 - 0s - loss: 0.1845 - accuracy: 0.9356 - val_loss: 0.3411 - val_accuracy: 0.8770 - 468ms/epoch - 10ms/step
Epoch 5/20
46/46 - 0s - loss: 0.1631 - accuracy: 0.9430 - val_loss: 0.3482 - val_accuracy: 0.8775 - 466ms/epoch - 10ms/step
Epoch 6/20
46/46 - 0s - loss: 0.1378 - accuracy: 0.9535 - val_loss: 0.3105 - val_accuracy: 0.8855 - 466ms/epoch - 10ms/step
Epoch 7/20
46/46 - 1s - loss: 0.1241 - accuracy: 0.9572 - val_loss: 0.4187 - val_accuracy: 0.8695 - 505ms/epoch - 11ms/step
Epoch 8/20
46/46 - 1s - loss: 0.1054 - accuracy: 0.9640 - val_loss: 0.3438 - val_accuracy: 0.8825 - 508ms/epoch - 11ms/step
Epoch 9/20
46/46 - 0s 

The `history` object has a `metrics` attribute that records both the training and validation accuracy at each epoch. We'll wrangle it into long-format for plotting with Altair.

In [15]:
_ = (
    pd.DataFrame(history.history)
    .reset_index()
    .rename(columns={"index": "epoch"})
    .assign(epoch=lambda df: df.epoch + 1)
)
df_mlp = pd.concat(
    [
        _.iloc[:, 0:3].assign(fold="training"),
        _.iloc[:, [0, -2, -1]]
        .rename(columns={"val_accuracy": "accuracy", "val_loss": "loss"})
        .assign(fold="validation"),
    ],
    axis=0,
).reset_index(drop=True)
plot_mlp = (
    alt.Chart(df_mlp)
    .mark_line(point=True)
    .encode(
        x="epoch:Q",
        y="accuracy",
        color="fold",
        tooltip=["epoch", alt.Tooltip("accuracy", format=",.2f")],
    )
)
plot_mlp

alt.Chart(...)

To compute the test accuracy, we rerun the entire sequence above, replacing the last line with

In [13]:
history_test = model.fit(X_train_1h[mask,:], y_train[mask],
          batch_size=512, epochs=20,
          verbose=2,
          validation_data=(X_test_1h, y_test))

Epoch 1/20
46/46 - 1s - loss: 0.0223 - accuracy: 0.9948 - val_loss: 0.7094 - val_accuracy: 0.8551 - 806ms/epoch - 18ms/step
Epoch 2/20
46/46 - 1s - loss: 0.0066 - accuracy: 0.9991 - val_loss: 0.7450 - val_accuracy: 0.8534 - 730ms/epoch - 16ms/step
Epoch 3/20
46/46 - 1s - loss: 0.0132 - accuracy: 0.9970 - val_loss: 0.7840 - val_accuracy: 0.8543 - 699ms/epoch - 15ms/step
Epoch 4/20
46/46 - 1s - loss: 0.0145 - accuracy: 0.9964 - val_loss: 0.8140 - val_accuracy: 0.8531 - 694ms/epoch - 15ms/step
Epoch 5/20
46/46 - 1s - loss: 0.0131 - accuracy: 0.9971 - val_loss: 0.8322 - val_accuracy: 0.8528 - 739ms/epoch - 16ms/step
Epoch 6/20
46/46 - 1s - loss: 0.0142 - accuracy: 0.9966 - val_loss: 0.8511 - val_accuracy: 0.8516 - 726ms/epoch - 16ms/step
Epoch 7/20
46/46 - 1s - loss: 0.0023 - accuracy: 0.9998 - val_loss: 0.9198 - val_accuracy: 0.8513 - 693ms/epoch - 15ms/step
Epoch 8/20
46/46 - 1s - loss: 0.0176 - accuracy: 0.9966 - val_loss: 0.9492 - val_accuracy: 0.8508 - 691ms/epoch - 15ms/step
Epoch 9/